In [0]:
import numpy as np 
import pandas as pd 

In [2]:
books_data=pd.read_csv('/content/books.csv')
books_data

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,7130616,7130616,7392860,19,441019455,9.780441e+12,Ilona Andrews,2010.0,Bayou Moon,"Bayou Moon (The Edge, #2)",eng,4.09,17204,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...
9996,9997,208324,208324,1084709,19,067973371X,9.780680e+12,Robert A. Caro,1990.0,Means of Ascent,"Means of Ascent (The Years of Lyndon Johnson, #2)",eng,4.25,12582,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...
9997,9998,77431,77431,2393986,60,039330762X,9.780393e+12,Patrick O'Brian,1977.0,The Mauritius Command,The Mauritius Command,eng,4.35,9421,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...
9998,9999,8565083,8565083,13433613,7,61711527,9.780062e+12,Peggy Orenstein,2011.0,Cinderella Ate My Daughter: Dispatches from th...,Cinderella Ate My Daughter: Dispatches from th...,eng,3.65,11279,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...


In [3]:
ratings_data = pd.read_csv('/content/ratings.csv')
ratings_data

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
...,...,...,...
981751,10000,48386,5
981752,10000,49007,4
981753,10000,49383,5
981754,10000,50124,5


In [4]:
#Checking to see if the feature 'book_id' and 'best_book_id' are similar.
books_data['check']=books_data.apply(lambda x:int(x['book_id'] == x['best_book_id']),axis=1)
books_data[books_data['check']==0]

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,check
29,30,8442457,19288043,13306276,196,297859382,9.780298e+12,Gillian Flynn,2012.0,Gone Girl,Gone Girl,eng,4.03,512475,1626519,121614,38874,80807,280331,616031,610476,https://images.gr-assets.com/books/1339602131m...,https://images.gr-assets.com/books/1339602131s...,0
32,33,930,929,1558965,220,739326228,9.780739e+12,Arthur Golden,1997.0,Memoirs of a Geisha,Memoirs of a Geisha,eng,4.08,1300209,1418172,25605,23500,59033,258700,517157,559782,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,0
37,38,14050,18619684,2153746,167,965818675,9.780966e+12,Audrey Niffenegger,2003.0,The Time Traveler's Wife,The Time Traveler's Wife,eng,3.95,746287,1308667,43382,44339,85429,257805,427210,493884,https://images.gr-assets.com/books/1437728815m...,https://images.gr-assets.com/books/1437728815s...,0
53,54,11,386162,3078186,257,345391802,9.780345e+12,Douglas Adams,1979.0,The Hitchhiker's Guide to the Galaxy,The Hitchhiker's Guide to the Galaxy (Hitchhik...,en-US,4.20,936782,1006479,20345,21764,41962,145173,299579,498001,https://images.gr-assets.com/books/1327656754m...,https://images.gr-assets.com/books/1327656754s...,0
83,84,7677,6424171,3376836,176,030734813X,9.780307e+12,Michael Crichton,1990.0,Jurassic Park,"Jurassic Park (Jurassic Park, #1)",spa,3.96,447833,650483,8143,15736,34222,138327,231583,230615,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9776,9777,8428984,8428985,13292213,10,NaN,NaN,Hugh Howey,2010.0,NaN,Half Way Home,en-GB,3.70,5012,10231,787,161,799,3139,3942,2190,https://images.gr-assets.com/books/1374845830m...,https://images.gr-assets.com/books/1374845830s...,0
9786,9787,18271235,20385637,25734842,12,1476738963,9.781477e+12,Laura McBride,2014.0,We Are Called to Rise,We Are Called to Rise,eng,3.94,5687,9458,1555,100,389,2094,4241,2634,https://images.gr-assets.com/books/1403347563m...,https://images.gr-assets.com/books/1403347563s...,0
9892,9893,16081202,17978160,21880142,11,NaN,NaN,Emery Lord,2014.0,Open Road Summer,Open Road Summer,eng,3.91,5017,11408,1335,271,674,2663,4002,3798,https://images.gr-assets.com/books/1405104324m...,https://images.gr-assets.com/books/1405104324s...,0
9898,9899,25813921,27213329,45670634,32,1442397144,9.781442e+12,Angela Duckworth,2016.0,"Grit: Passion, Perseverance, and the Science o...","Grit: Passion, Perseverance, and the Science o...",eng,4.08,7971,17123,1634,303,657,3023,6540,6600,https://images.gr-assets.com/books/1458533580m...,https://images.gr-assets.com/books/1458533580s...,0


In [5]:
books_data['book_id'].isnull().sum()


0

In [6]:
books_data['best_book_id'].isnull().sum()

0

In [0]:
#Dropping one feature which have same values as other feature like (title , orignal title ) and  (book_id,best_book_id)
books_data=books_data.drop(['title','best_book_id'],axis=1)

#Dropping features that cannot contribute in the recommendations
books_data=books_data.drop(['image_url','small_image_url','isbn','isbn13'],axis=1)


In [0]:
# Building the item-item similarity matrix 
booksDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0;
testdf = ratings_data
testdf=testdf[['user_id','rating']].groupby(testdf['book_id'])
for groupKey in testdf.groups.keys():
    tempDict={}

    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    booksDictonaries.append(tempDict)

In [0]:
from sklearn.feature_extraction import DictVectorizer
#Encoding the data using DictVectorizer
dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(booksDictonaries)

In [0]:
# Calculation of  the cosine similarity between the item vectors.
from sklearn.metrics.pairwise import cosine_similarity
pairwiseSimilarity = cosine_similarity(vector)

In [0]:
#Helper functions for printing input book and recommendations
def bookDetails(bookID):
    print("Book Title  :", books_data[books_data['id']==bookID]['original_title'].values[0])
    print("Book Author :",books_data[books_data['id']==bookID]['authors'].values[0])
    print("Book ID     :",bookID)
    print("\n")

def topRecommandations(bookID):
    row = reverseIndexMap[bookID]
    print("------ BOOK--------------------")
    bookDetails(bookID)
    print("-------RECOMMENDATIONS----------")
    similarBookIDs = [bookDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-7:-2][::-1]]

In [23]:
topRecommandations(1245)

------ BOOK--------------------
Book Title  : The Brethren
Book Author : John Grisham
Book ID     : 1245


-------RECOMMENDATIONS----------
Book Title  : The Street Lawyer
Book Author : John Grisham
Book ID     : 1037


Book Title  : The Last Juror
Book Author : John Grisham
Book ID     : 1388


Book Title  : The Testament
Book Author : John Grisham
Book ID     : 1057


Book Title  : The Broker
Book Author : John Grisham
Book ID     : 1301


Book Title  : The King of Torts
Book Author : John Grisham
Book ID     : 1407


